# My First Query

One of the most powerful features of Marvin 2.0 is ability to query the newly created DRP and DAP databases. You can do this in two ways:
1. via the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)** or
2. via Python (in the terminal/notebook/script) with **Marvin-tools**.

The best part is that both interfaces use the same underlying query structure, so your input search will be the same. Here we will run a few queries with Marvin-tools to learn the basics of how to construct a query and also test drive some of the more advanced features that are unique to the Marvin-tools version of querying.

In [1]:
# Python 2/3 compatibility
from __future__ import print_function, division, absolute_import

In [5]:
from marvin import config
config.mode = 'remote'
config.setRelease('MPL-4')

In [6]:
from marvin.tools.query import Query

Let's search for galaxies with M$_\star$ > 3 $\times$ 10$^{11}$ M$_\odot$.

To specify our search parameter, M$_\star$, we must know the database table and name of the parameter. In this case, MaNGA uses the NASA-Sloan Atlas (NSA) for target selection so we will use the Sersic profile determination for stellar mass, which is the **`sersic_mass`** parameter of the **`nsa`** table, so our search parameter will be **`nsa.sersic_mass`**.  You can also use **`nsa.sersic_logmass`**

Generically, the search parameter will take the form **`table.parameter`**.

In [8]:
myquery1 = 'nsa.sersic_mass > 3e11'
# or
myquery1 = 'nsa.sersic_logmass > 11.47'

In [9]:
q1 = Query(searchfilter=myquery1)
r1 = q1.run()

         Error: HTTPSConnectionPool(host='api.sdss.org', port=443): Max retries
         exceeded with url: /marvin2/api/general/getroutemap?release=MPL-4
         (Caused by NewConnectionError('<requests.packages.urllib3.connection.Ve
         rifiedHTTPSConnection object at 0x11a197310>: Failed to establish a new
         connection: [Errno 8] nodename nor servname provided, or not known',))


MarvinError: No URL Map found.  Cannot make remote calls!

Running the query produces a **`Results`** object (**`r1`**):

In [ ]:
# show results
r1.results

We will learn how to use the features of our **`Results`** object a little bit later, but first let's revise our search to see how more complex search queries work.

## Multiple Search Criteria

Let's add to our search to find only galaxies with a redshift less than 0.1.

Redshift is the **`z`** parameter and is also in the **`nsa`** table, so its full search parameter designation is **`nsa.z`**.

In [ ]:
myquery2 = 'nsa.sersic_mass > 3e11 AND nsa.z < 0.1'

In [ ]:
q2 = Query(searchfilter=myquery2)
r2 = q2.run()

In [ ]:
r2.results

## Compound Search Statements

We were hoping for a few more than 3 galaxies, so let's try to increase our search by broadening the criteria to also include galaxies with 127 fiber IFUs and a b/a ratio of at least 0.95.

To find 127 fiber IFUs, we'll use the **`name`** parameter of the **`ifu`** table, which means the full search parameter is **`ifu.name`**. However, **`ifu.name`** returns the IFU design name, such as `12701`, so we need to to set the value to `127*`.

The b/a ratio is in **`nsa`** table as the **`ba90`** parameter.

We're also going to join this to or previous query with an **`OR`** operator and use parentheses to group our individual search statements into a compound search statement.

In [ ]:
myquery3 = '(nsa.sersic_mass > 3e11 AND nsa.z < 0.1) OR (ifu.name=127* AND nsa.ba90 >= 0.95)'

In [ ]:
q3 = Query(searchfilter=myquery3)
r3 = q3.run()

In [ ]:
r3.results

## Design Your Own Search

OK, now it's your turn to try designing a search.

**Exercise**: Write a search filter that will find galaxies with a redshift less than 0.02 that were observed with the 1901 IFU?

In [ ]:
# Enter your search here

You should get 8 results:

`[NamedTuple(mangaid='1-22438', plate=7992, name='1901', z=0.016383046284318),
 NamedTuple(mangaid='1-113520', plate=7815, name='1901', z=0.0167652331292629),
 NamedTuple(mangaid='1-113698', plate=8618, name='1901', z=0.0167444702237844),
 NamedTuple(mangaid='1-134004', plate=8486, name='1901', z=0.0185601413249969),
 NamedTuple(mangaid='1-155903', plate=8439, name='1901', z=0.0163660924881697),
 NamedTuple(mangaid='1-167079', plate=8459, name='1901', z=0.0157109703868628),
 NamedTuple(mangaid='1-209729', plate=8549, name='1901', z=0.0195561610162258),
 NamedTuple(mangaid='1-277339', plate=8254, name='1901', z=0.0192211158573627)]`

## Finding the Available Parameters
Now you might want to go out and try all of the interesting queries that you've been saving up, but you don't know what the parameters are called or what database table they are in. 

You can find all of the availabale parameters by:
1. clicking on in the **Return Parameters** dropdown menu on the left side of the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)**,
2. reading the **[Marvin Docs page](https://api.sdss.org/doc/manga/marvin/refactor/query_params.html#marvin-query-parameters)**, or
3. via **Marvin-tools** (see next two cells)

In [ ]:
# You might have to do an svn update to get this to work (otherwise try the next cell)
q = Query()
q.get_available_params()

In [ ]:
# try this if the previous cell didn't return a list of parameters
from marvin.api.api import Interaction
from pprint import pprint
url = config.urlmap['api']['getparams']['url']
ii = Interaction(route=url)
mykeys = ii.getData()
pprint(mykeys)

Go ahead and try to create some new searches on your own from the parameter list. Please feel free to also try out the some of the same search on the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)**.

## Returning Bonus Parameters

Often you want to run a query and see the value of parameters that you didn't explicitly search on. For instance, you want to find galaxies above a redshift of 0.1 and would like to know their RA and DECs.

In Marvin-tools, this is as easy as specifying the **`returnparams`** option with either a string (for a single bonus parameter) or a list of strings (for multiple bonus parameters).

In [ ]:
myquery5 = 'nsa.z > 0.1'
bonusparams5 = ['cube.ra', 'cube.dec']
# bonusparams5 = 'cube.ra'  # This works too

q5 = Query(searchfilter=myquery5, returnparams=bonusparams5)
r5 = q5.run()

In [ ]:
r5.results

Next time, we'll take a closer look at the **`Results`** class and its built in MaNGA convenience functions.

## Useful Resources

Check out these pages on the Marvin Docs site for more **`Query`** tips and tricks.

- **[Query](https://api.sdss.org/doc/manga/marvin/refactor/query.html)**
- **[Example Queries](https://api.sdss.org/doc/manga/marvin/refactor/tutorials/query-examples.html)**
- **[Results](https://api.sdss.org/doc/manga/marvin/refactor/results.htmlpages)** 
- **[Boolean Search Tutorial](https://api.sdss.org/doc/manga/marvin/refactor/tutorials/boolean-search-tutorial.html)**